In [32]:
import os
import numpy
import prophet
import pandas as pd
from IPython.display import clear_output
import logging

#Dataset from: https://finance.yahoo.com/quote/VOO/history

logging.getLogger('prophet').setLevel(logging.WARNING)


In [33]:
data_import = pd.read_csv('VOO.csv', parse_dates = ["Date"])
data_import

,Date,Open,High,Low,Close,Adj Close,Volume
0,2022-11-09,349.079987,350.220001,343.359985,343.869995,338.203125,3570800
1,2022-11-10,356.519989,363.010010,354.410004,362.690002,356.713043,4603600
2,2022-11-11,363.690002,367.019989,361.739990,366.160004,360.125824,4877400
3,2022-11-14,364.570007,367.790009,362.890015,363.220001,357.234283,5274300
4,2022-11-15,368.649994,369.739990,362.589996,366.220001,360.184845,4597800
...,...,...,...,...,...,...,...
246,2023-11-02,391.929993,395.910004,391.920013,395.779999,395.779999,4752200
247,2023-11-03,397.920013,400.850006,397.839996,399.440002,399.440002,5980300
248,2023-11-06,400.119995,400.730011,398.459991,400.209991,400.209991,4007100
249,2023-11-07,400.260010,402.040009,399.230011,401.339996,401.339996,3358600


In [34]:
day_mapper = {0 : "Monday", 1 : "Tuesday", 2 : "Wednesday", 3 : "Thursday", 4 : "Friday", 5 : "Saturday", 6 : "Sunday" }
data_import["DayOfWeek"] = data_import["Date"].map(lambda x: day_mapper[x.dayofweek])
data_import

,Date,Open,High,Low,Close,Adj Close,Volume,DayOfWeek
0,2022-11-09,349.079987,350.220001,343.359985,343.869995,338.203125,3570800,Wednesday
1,2022-11-10,356.519989,363.010010,354.410004,362.690002,356.713043,4603600,Thursday
2,2022-11-11,363.690002,367.019989,361.739990,366.160004,360.125824,4877400,Friday
3,2022-11-14,364.570007,367.790009,362.890015,363.220001,357.234283,5274300,Monday
4,2022-11-15,368.649994,369.739990,362.589996,366.220001,360.184845,4597800,Tuesday
...,...,...,...,...,...,...,...,...
246,2023-11-02,391.929993,395.910004,391.920013,395.779999,395.779999,4752200,Thursday
247,2023-11-03,397.920013,400.850006,397.839996,399.440002,399.440002,5980300,Friday
248,2023-11-06,400.119995,400.730011,398.459991,400.209991,400.209991,4007100,Monday
249,2023-11-07,400.260010,402.040009,399.230011,401.339996,401.339996,3358600,Tuesday


Method 1: Average Dip Per Day


.diff() function


In [35]:
data_import["Open"].diff()

0           NaN
1      7.440002
2      7.170013
3      0.880005
4      4.079987
         ...   
246    6.839997
247    5.990020
248    2.199982
249    0.140015
250    1.729980
Name: Open, Length: 251, dtype: float64

In [36]:
data_import['diff_from_previous_day'] = (data_import["Open"].diff() / data_import["Open"]) * 100
data_import

,Date,Open,High,Low,Close,Adj Close,Volume,DayOfWeek,diff_from_previous_day
0,2022-11-09,349.079987,350.220001,343.359985,343.869995,338.203125,3570800,Wednesday,NaN
1,2022-11-10,356.519989,363.010010,354.410004,362.690002,356.713043,4603600,Thursday,2.086840
2,2022-11-11,363.690002,367.019989,361.739990,366.160004,360.125824,4877400,Friday,1.971463
3,2022-11-14,364.570007,367.790009,362.890015,363.220001,357.234283,5274300,Monday,0.241382
4,2022-11-15,368.649994,369.739990,362.589996,366.220001,360.184845,4597800,Tuesday,1.106737
...,...,...,...,...,...,...,...,...,...
246,2023-11-02,391.929993,395.910004,391.920013,395.779999,395.779999,4752200,Thursday,1.745209
247,2023-11-03,397.920013,400.850006,397.839996,399.440002,399.440002,5980300,Friday,1.505333
248,2023-11-06,400.119995,400.730011,398.459991,400.209991,400.209991,4007100,Monday,0.549831
249,2023-11-07,400.260010,402.040009,399.230011,401.339996,401.339996,3358600,Tuesday,0.034981


In [37]:
data_import.groupby("DayOfWeek")["diff_from_previous_day"].mean()

DayOfWeek
Friday       0.021093
Monday       0.128853
Thursday    -0.000763
Tuesday      0.217858
Wednesday   -0.097203
Name: diff_from_previous_day, dtype: float64

In [38]:
dates = pd.date_range(start=data_import["Date"].min(), end = data_import["Date"].max())

#Add the name of the day of the week
date_table = pd.DataFrame(data = {"Calendar Date": dates})
date_table["Weekday"] = date_table["Calendar Date"].map(lambda x: day_mapper[x.dayofweek])
date_table

,Calendar Date,Weekday
0,2022-11-09,Wednesday
1,2022-11-10,Thursday
2,2022-11-11,Friday
3,2022-11-12,Saturday
4,2022-11-13,Sunday
...,...,...
360,2023-11-04,Saturday
361,2023-11-05,Sunday
362,2023-11-06,Monday
363,2023-11-07,Tuesday


In [39]:
data_import.to_clipboard()

In [40]:
full_calendar = pd.merge(left = date_table, right = data_import, how="left", left_on = 'Calendar Date', right_on = "Date")
full_calendar = full_calendar[~full_calendar["Weekday"].isin(["Saturday", "Sunday"])]
full_calendar.rename(columns = {"Date": 'Trading Day'}, inplace = True)
full_calendar

,Calendar Date,Weekday,Trading Day,Open,High,Low,Close,Adj Close,Volume,DayOfWeek,diff_from_previous_day
0,2022-11-09,Wednesday,2022-11-09,349.079987,350.220001,343.359985,343.869995,338.203125,3570800.0,Wednesday,NaN
1,2022-11-10,Thursday,2022-11-10,356.519989,363.010010,354.410004,362.690002,356.713043,4603600.0,Thursday,2.086840
2,2022-11-11,Friday,2022-11-11,363.690002,367.019989,361.739990,366.160004,360.125824,4877400.0,Friday,1.971463
5,2022-11-14,Monday,2022-11-14,364.570007,367.790009,362.890015,363.220001,357.234283,5274300.0,Monday,0.241382
6,2022-11-15,Tuesday,2022-11-15,368.649994,369.739990,362.589996,366.220001,360.184845,4597800.0,Tuesday,1.106737
...,...,...,...,...,...,...,...,...,...,...,...
358,2023-11-02,Thursday,2023-11-02,391.929993,395.910004,391.920013,395.779999,395.779999,4752200.0,Thursday,1.745209
359,2023-11-03,Friday,2023-11-03,397.920013,400.850006,397.839996,399.440002,399.440002,5980300.0,Friday,1.505333
362,2023-11-06,Monday,2023-11-06,400.119995,400.730011,398.459991,400.209991,400.209991,4007100.0,Monday,0.549831
363,2023-11-07,Tuesday,2023-11-07,400.260010,402.040009,399.230011,401.339996,401.339996,3358600.0,Tuesday,0.034981


In [41]:
full_calendar.to_clipboard()

In [42]:
full_calendar = full_calendar.bfill(axis = "rows").reset_index(drop = True)

In [43]:
def day_backtester(day, amount_to_invest, data):
    temp_data = data[data["Weekday"] == day]
    temp_data ["Shares Owned"] = amount_to_invest / temp_data["Open"]
    final_price = temp_data["Open"].iloc[-1]
    final_amount = temp_data["Shares Owned"].sum() *final_price
    formatted_final_amount = "${:,.2f}".format(final_amount)
    return formatted_final_amount

In [44]:
for i in full_calendar["Weekday"].unique():
    print(i, day_backtester(i, 750, full_calendar))

Wednesday $41,769.24
Thursday $39,984.16
Friday $40,589.74
Monday $40,747.59
Tuesday $40,690.04


C:\Users\zakbh\AppData\Local\Temp\ipykernel_24272\3785564227.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_data ["Shares Owned"] = amount_to_invest / temp_data["Open"]
C:\Users\zakbh\AppData\Local\Temp\ipykernel_24272\3785564227.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_data ["Shares Owned"] = amount_to_invest / temp_data["Open"]
C:\Users\zakbh\AppData\Local\Temp\ipykernel_24272\3785564227.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFr